In [1]:
import os
import random
from moviepy.editor import *
import moviepy.audio.fx.all as afx
from moviepy.video import fx
from moviepy.config import change_settings
change_settings({"FFMPEG_BINARY":"ffmpeg"})

import whisper

# from PIL import Image

In [2]:
select_number_of_news = int(input('Select news: '))
select_number_of_news

9

In [3]:
select_lang = int(input('Select language: '))

if select_lang == 1:
    folder_lang = 'english'
    lang = 'en'
elif select_lang == 2:
    folder_lang = 'spanish'
    lang = 'es'
else:
    pass

In [4]:
# Configuración
carpeta1 = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/3_extract_img_videos/img_by_number/{select_number_of_news}'
carpeta2 = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/3_extract_img_videos/video_by_number/{select_number_of_news}'
audio_musica = f'/mnt/D8E84E4DE84E2A58/backUp_Img_Vid/Intro/History/New_metal.mp3'   # Ruta específica del audio de música
audio_voz = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/2_create_final_audio/output_{lang}_b/{select_number_of_news}/audio_final_{select_number_of_news}.wav' 

In [5]:
# Transcribir el audio usando Whisper
print("Transcribiendo audio con Whisper...")
modelo_whisper = whisper.load_model("base")  # Puedes usar "small", "medium", o "large" para mayor precisión
resultados = modelo_whisper.transcribe(audio_voz, language=lang)

# Extraer segmentos de texto con sus tiempos
segmentos = resultados['segments']


print("Transcripción completada.")

Transcribiendo audio con Whisper...
Transcripción completada.


In [6]:
# Configuración global de texto
FONT_SIZE = 50
MAX_TEXT_WIDTH = 1000  # Ancho máximo del texto
LINE_SPACING = 10  # Espaciado entre líneas
MAX_LINES = 4  # Máximo de líneas visibles

In [7]:
# Función para ajustar tamaño del clip
def ajustar_clip(clip):
    clip_resized = clip.resize(height=1920) if clip.h / clip.w > 1920 / 1080 else clip.resize(width=1080)
    return clip_resized.on_color(
        size=(1080, 1920),
        color=(0, 0, 0),
        pos=("center", "center")
    )


In [8]:
# Función para agregar texto sincronizado con tamaño consistente
def agregar_texto_sincronizado(clip, texto, duracion):
    # Tamaño de fuente fijo
    fontsize = 50
    # Crear el text clip con ajuste automático dentro del área definida
    text_clip = TextClip(
        texto,
        fontsize=fontsize,
        color="white",
        font="Arial-Bold",
        bg_color="black",
        stroke_color="black",
        stroke_width=2,
        method="caption",  # Ajustar texto con saltos de línea automáticos
        size=(1000, None)  # Ancho máximo para el texto, altura automática
    ).set_duration(duracion).set_position(("center", 1300))  # Mueve el texto más arriba (ajusta 1600 según necesidad)
    return CompositeVideoClip([clip, text_clip])


In [9]:
# Seleccionar archivos aleatorios
def selecciona_archivos(carpeta, num_archivos=25):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(('.jpg', '.png', '.mp4', '.avi'))]
    if not archivos:
        raise ValueError(f"No hay archivos válidos en {carpeta}")
    return random.sample(archivos, min(num_archivos, len(archivos)))

In [10]:
carpeta1_archivos = selecciona_archivos(carpeta1)
carpeta2_archivos = selecciona_archivos(carpeta2)
archivos = carpeta1_archivos + carpeta2_archivos
random.shuffle(archivos)

In [11]:
# Crear clips sincronizados
clips_seleccionados = []
for archivo, segmento in zip(archivos, segmentos):
    archivo_path = os.path.join(carpeta1 if archivo in carpeta1_archivos else carpeta2, archivo)
    texto = segmento['text']
    duracion = segmento['end'] - segmento['start']
    try:
        if archivo.endswith(('.jpg', '.png')):
            clip = ImageClip(archivo_path).set_duration(duracion)
        elif archivo.endswith(('.mp4', '.avi')):
            clip = VideoFileClip(archivo_path).subclip(0, min(duracion, VideoFileClip(archivo_path).duration))
        else:
            continue
        clip = ajustar_clip(clip)
        clip_con_texto = agregar_texto_sincronizado(clip, texto, duracion)
        clips_seleccionados.append(clip_con_texto)
    except Exception as e:
        print(f"Error al procesar el archivo {archivo}: {e}")

In [12]:
# Ajustar duración total del video
audio_principal_voz = AudioFileClip(audio_voz)
duracion_audio_principal = audio_principal_voz.duration
suma_duracion_clips = sum([clip.duration for clip in clips_seleccionados])

In [13]:
if suma_duracion_clips > duracion_audio_principal:
    duracion_acumulada = 0
    clips_finales = []
    for clip in clips_seleccionados:
        if duracion_acumulada + clip.duration > duracion_audio_principal:
            clip = clip.subclip(0, duracion_audio_principal - duracion_acumulada)
        duracion_acumulada += clip.duration
        clips_finales.append(clip)
        if duracion_acumulada >= duracion_audio_principal:
            break
else:
    factor_ajuste = duracion_audio_principal / suma_duracion_clips
    clips_finales = [clip.set_duration(clip.duration * factor_ajuste) for clip in clips_seleccionados]





In [14]:
# Combinar audio principal con música de fondo
try:
    audio_fondo = AudioFileClip(audio_musica)
    if audio_fondo.duration < duracion_audio_principal:
        n_repeats = int(duracion_audio_principal // audio_fondo.duration) + 1
        audio_fondo = concatenate_audioclips([audio_fondo] * n_repeats)
    audio_fondo = audio_fondo.subclip(0, duracion_audio_principal).volumex(0.1)
    final_audio = CompositeAudioClip([audio_fondo, audio_principal_voz.volumex(1.8)])
except Exception as e:
    print(f"Error al combinar los audios: {e}")
    final_audio = audio_principal_voz

In [15]:
# Crear y guardar el video final
try:
    final_video = concatenate_videoclips(clips_finales, method="compose").set_audio(final_audio)
    final_video.write_videofile(f'videos_by_news_short/{select_number_of_news}.mp4', codec="h264_nvenc", audio_codec="aac", fps=24)
    print("¡Video creado con éxito!")
except Exception as e:
    print(f"Error al guardar el video: {e}")

Moviepy - Building video videos_by_news_short/9.mp4.
MoviePy - Writing audio in 9TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video videos_by_news_short/9.mp4



Moviepy - Done !
Moviepy - video ready videos_by_news_short/9.mp4
¡Video creado con éxito!
